In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import sklearn.metrics as sk

import torch

# Confirm that the GPU is detected

assert torch.cuda.is_available()

# Get the GPU device name.
device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")
device = torch.device("cuda")

Mounted at /content/drive
Found device: Tesla T4, n_gpu: 1


In [ ]:
!pip install transformers
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
print('success!')

import os
import zipfile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 95.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.9 MB/s eta 0:00:00
success!


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertTokenizer, RobertaTokenizer, XLNetModel, XLNetTokenizer
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import sys
import numpy as np
import time
import datetime

def tokenize_and_format_bert(sentences, model_type):
  tokenizer = BertTokenizer.from_pretrained(model_type, do_lower_case=True)

  # Tokenize all of the sentences and map the tokens to thier word IDs.
  input_ids = []
  attention_masks = []

  # For every sentence...
  for sentence in sentences:
      encoded_dict = tokenizer.encode_plus(
                          sentence,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          max_length = 512,           # Pad & truncate all sentences.
                          padding = 'max_length',
                          truncation = True,
                          return_attention_mask = True,   # Construct attn. masks.
                          return_tensors = 'pt',     # Return pytorch tensors.
                    )

      input_ids.append(encoded_dict['input_ids'])
      attention_masks.append(encoded_dict['attention_mask'])

  return input_ids, attention_masks

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
# from transformers import RobertaTokenizer
def tokenize_and_format_roberta(sentences, model_type):
  tokenizer = RobertaTokenizer.from_pretrained(model_type, max_length = 512)

  input_ids = []
  attention_masks = []

  # For every sentence...
  for sentence in sentences:
      encoded_dict = tokenizer.encode_plus(
                          sentence,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          max_length = 512,           # Pad & truncate all sentences.
                          padding = 'max_length',
                          truncation = True,
                          return_attention_mask = True,   # Construct attn. masks.
                          return_tensors = 'pt',     # Return pytorch tensors.
                    )

      # Add the encoded sentence to the list.
      input_ids.append(encoded_dict['input_ids'])

      # And its attention mask (simply differentiates padding from non-padding).
      attention_masks.append(encoded_dict['attention_mask'])
  return input_ids, attention_masks

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from transformers import XLNetModel, XLNetTokenizer
from transformers import AdamW

from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
def tokenize_and_format_xlnet(sentences, model_type):
  tokenizer = XLNetTokenizer.from_pretrained(model_type, do_lower_case=True)

  input_ids = []
  attention_masks = []

  # For every sentence...
  for sentence in sentences:
      encoded_dict = tokenizer.encode_plus(
                          sentence,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          max_length = 512,           # Pad & truncate all sentences.
                          padding = 'max_length',
                          truncation = True,
                          return_attention_mask = True,   # Construct attn. masks.
                          return_tensors = 'pt',     # Return pytorch tensors.
                    )

      # Add the encoded sentence to the list.
      input_ids.append(encoded_dict['input_ids'])

      # And its attention mask (simply differentiates padding from non-padding).
      attention_masks.append(encoded_dict['attention_mask'])
  return input_ids, attention_masks

In [ ]:
def convert_to_tensors(input_ids, attention_masks):
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  return input_ids, attention_masks

In [ ]:
import pandas as pd

df = pd.read_csv('final_dataset_cleaned.csv')

df = df.sample(frac=1).reset_index(drop=True)

texts = df.text.values
label = df.label.values

### tokenize_and_format
bert_input_ids, bert_attention_masks = tokenize_and_format_bert(texts, 'bert-base-uncased')
# Convert the lists into tensors.
bert_input_ids, bert_attention_masks = convert_to_tensors(bert_input_ids, bert_attention_masks)
labels = torch.tensor(label)

# Print sentence 0, now as a list of IDs.
print('Original: ', texts[0])
print('BERT Token IDs:', bert_input_ids[0])

Original:  dysphoric mania i was diagnosed with bipolar 1 around two years ago, &amp; my mania seems to be dysphoric mania. i'm jumpy and super active, but also slow. I have no need for sleep. i'm super irritable. i'm angry. I have low self esteem. i'm bored constantly, nothing makes me feel fulfilled. I speak so fast people usually can't understand me. i'm on the go constantly. i'm extremely happy and in a good mood but also have feelings of hopelessness. I have erratic behavior. I spend a shit ton of money. i'm hyper sexual, etc. my episodes are very mixed and i'm currently going through a severe episode because my medications have been jacked up completely by my psychiatrist, and I just need someone to vent to and talk to that understands what i'm going through because I feel like i'm losing my mind. I have people telling me i'm depressed, people telling me i'm manic, and then me feeling like i'm completely fine and nothing is wrong and now I feel like i'm faking my bipolar 1 diagno

# BERT Large

In [ ]:
import pandas as pd

df = pd.read_csv('final_dataset_cleaned.csv')

df = df.sample(frac=1).reset_index(drop=True)

texts = df.text.values
label = df.label.values

### tokenize_and_format
bert_large_input_ids, bert_large_attention_masks = tokenize_and_format_bert(texts, 'bert-large-uncased')
# Convert the lists into tensors.
bert_large_input_ids, bert_large_attention_masks = convert_to_tensors(bert_large_input_ids, bert_large_attention_masks)
labels = torch.tensor(label)

# Print sentence 0, now as a list of IDs.
print('Original: ', texts[0])
print('BERT Large Token IDs:', bert_large_input_ids[0])

In [ ]:
import pandas as pd

df = pd.read_csv('final_dataset_cleaned.csv')

df = df.sample(frac=1).reset_index(drop=True)

texts = df.text.values
label = df.label.values

### tokenize_and_format
roberta_input_ids, roberta_attention_masks = tokenize_and_format_roberta(texts, 'roberta-base')

# Convert the lists into tensors.
roberta_input_ids, roberta_attention_masks = convert_to_tensors(roberta_input_ids, roberta_attention_masks)
labels = torch.tensor(label)

# Print sentence 0, now as a list of IDs.
print('Original: ', texts[0])
print('ROBERTA Token IDs:', roberta_input_ids[0])



Original:  can you also read people’s minds? i mean i don’t mean like magically viewing what’s going on thru someone’s mind. You get an intuition of it, you focus on the moment and eliminate details and there’s only thought patterns left. you observe people’s behavior and mimics. and there must be a timeline they are living on. you view their frequency. Am I delusional?
ROBERTA Token IDs: tensor([    0,  7424,    47,    67,  1166,    82,    17,    27,    29,  7283,
          116,   939,  1266,   939,   218,    17,    27,    90,  1266,   101,
        39408,  7603,    99,    17,    27,    29,   164,    15, 34876,   951,
           17,    27,    29,  1508,     4,   370,   120,    41, 39054,     9,
           24,     6,    47,  1056,    15,     5,  1151,     8,  7677,  1254,
            8,    89,    17,    27,    29,   129,   802,  8117,   314,     4,
           47, 14095,    82,    17,    27,    29,  3650,     8, 23684,  2857,
            4,     8,    89,   531,    28,    10, 10589,    51

In [ ]:
import pandas as pd

df = pd.read_csv('final_dataset_cleaned.csv')

df = df.sample(frac=1).reset_index(drop=True)

texts = df.text.values
label = df.label.values

### tokenize_and_format
xlnet_input_ids, xlnet_attention_masks = tokenize_and_format_xlnet(texts, 'xlnet-base-cased')

# Convert the lists into tensors.
xlnet_input_ids, xlnet_attention_masks = convert_to_tensors(xlnet_input_ids, xlnet_attention_masks)
labels = torch.tensor(label)

# Print sentence 0, now as a list of IDs.
print('Original: ', texts[0])
print('XLNET Token IDs:', xlnet_input_ids[0])



Original:  Not sure if bipolar mood swings or if actual feelings towards boyfriend throwaway account So every now and then I get this frustrated, resentful feeling towards my bf. We’ve been together 6 months. I keep telling myself it’s just intrusive thoughts/bipolar. He’s really nice to me, a great boyfriend so far. But I feel like he’s too into my business, like getting mad at me about finances when we don’t even live together or pay bills together. Also constantly telling me what to do/not do like if I wanna drink alcohol he tells me not to. I don’t like feeling like I have another parent, I get that he’s trying to be encouraging but it really bugs me. Also our lives are really boring together, we don’t do anything fun when we’re together, just lay around and go eat. He travels for work often and I’m jealous cause I’m stuck at home. He also wants me to move out of state while he’d still be going off to other countries anyway, which makes me resent him even more. Idk what to do or if

In [ ]:
def train_test_val_split(num_train, num_val, num_test, total, input_ids, attention_masks, labels):
  train_set = [(input_ids[i], attention_masks[i], labels[i]) for i in range(num_train)]
  val_set = [(input_ids[i], attention_masks[i], labels[i]) for i in range(num_train, num_val+num_train)]
  test_set = [(input_ids[i], attention_masks[i], labels[i]) for i in range(num_val + num_train, total)]
  return train_set, val_set, test_set

In [ ]:
total = len(df)

num_train = int(total * .8)
num_val = int(total * .1)
num_test = total - num_train - num_val

# make lists of 3-tuples (already shuffled the dataframe in cell above)

#BERT
bert_train_set, bert_val_set, bert_test_set = train_test_val_split(num_train, num_val, num_test, total, bert_input_ids, bert_attention_masks, labels)
bert_true_test_labels = [tup[2] for tup in bert_test_set]

train_text = [texts[i] for i in range(num_train)]
val_text = [texts[i] for i in range(num_train, num_val+num_train)]
test_text = [texts[i] for i in range(num_val + num_train, total)]

In [ ]:
total = len(df)

num_train = int(total * .8)
num_val = int(total * .1)
num_test = total - num_train - num_val

# make lists of 3-tuples (already shuffled the dataframe in cell above)

#ROBERT
roberta_train_set, roberta_val_set, roberta_test_set = train_test_val_split(num_train, num_val, num_test, total, roberta_input_ids, roberta_attention_masks, labels)
roberta_true_test_labels = [tup[2] for tup in roberta_test_set]

train_text = [texts[i] for i in range(num_train)]
val_text = [texts[i] for i in range(num_train, num_val+num_train)]
test_text = [texts[i] for i in range(num_val + num_train, total)]

In [ ]:
total = len(df)

num_train = int(total * .8)
num_val = int(total * .1)
num_test = total - num_train - num_val

# make lists of 3-tuples (already shuffled the dataframe in cell above)


#XLNET 
xlnet_train_set, xlnet_val_set, xlnet_test_set = train_test_val_split(num_train, num_val, num_test, total, xlnet_input_ids, xlnet_attention_masks, labels)
xlnet_true_test_labels = [tup[2] for tup in xlnet_test_set]

train_text = [texts[i] for i in range(num_train)]
val_text = [texts[i] for i in range(num_train, num_val+num_train)]
test_text = [texts[i] for i in range(num_val + num_train, total)]


In [ ]:
total = len(df)

num_train = int(total * .8)
num_val = int(total * .1)
num_test = total - num_train - num_val

# make lists of 3-tuples (already shuffled the dataframe in cell above)

#BERT Large
bert_large_train_set, bert_large_val_set, bert_large_test_set = train_test_val_split(num_train, num_val, num_test, total, bert_large_input_ids, bert_large_attention_masks, labels)
bert_large_true_test_labels = [tup[2] for tup in bert_test_set]

train_text = [texts[i] for i in range(num_train)]
val_text = [texts[i] for i in range(num_train, num_val+num_train)]
test_text = [texts[i] for i in range(num_val + num_train, total)]

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

bert_model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 10, # The number of output labels.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
bert_model.cuda()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

KeyboardInterrupt: ignored

In [ ]:
from transformers import AdamW
batch_size = 16
bert_optimizer = AdamW(bert_model.parameters(),
                  lr = 5e-6, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8 
                )
epochs = 3

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
import numpy as np
# function to get validation accuracy
def get_validation_performance2(val_set, model, batch_size, mode):
    batch_text = []
    batch_pred = []
    batch_labels = []
    # Put the model in evaluation mode
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0

    num_batches = int(len(val_set)/batch_size) + 1

    total_correct = 0

    for i in range(num_batches):
      end_index = min(batch_size * (i+1), len(val_set))

      batch = val_set[i*batch_size:end_index]
      if mode == 'train':
        batch_text = batch_text + val_text[i*batch_size:end_index].copy()
        batch_labels = batch_labels + [data[2].item() for data in batch]
      if len(batch)==0: continue
      input_id_tensors = torch.stack([data[0] for data in batch])
      input_mask_tensors = torch.stack([data[1] for data in batch])
      label_tensors = torch.stack([data[2] for data in batch])
      
      # Move tensors to the GPU
      b_input_ids = input_id_tensors.to(device)
      b_input_mask = input_mask_tensors.to(device)
      b_labels = label_tensors.to(device)
        
      # Tell pytorch not to bother with constructing the compute graph during
      # the forward pass, since this is only needed for backprop (training).
      with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        outputs = model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask,
                                labels=b_labels)
        loss = outputs.loss
        logits = outputs.logits
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        #Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the number of correctly labeled examples in batch
        pred_flat = np.argmax(logits, axis=1).flatten()
        batch_pred = batch_pred + pred_flat.tolist()
        labels_flat = label_ids.flatten()
        num_correct = np.sum(pred_flat == labels_flat)
        total_correct += num_correct

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_correct / len(val_set)
    return avg_val_accuracy, batch_text, batch_pred, batch_labels

In [ ]:
import random

# training loop

def train2(solver_object, train_set, val_set):
  
  model, optimizer, batch_size, epochs = solver_object

  # For each epoch...
  for epoch_i in range(epochs):
      # Perform one full pass over the training set.

      # print("")
      # print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
      # print('Training...')

      # Reset the total loss for this epoch.
      total_train_loss = 0

      # Put the model into training mode.
      model.train()

      # For each batch of training data...
      num_batches = int(len(train_set)/batch_size) + 1

      for i in range(num_batches):
        end_index = min(batch_size * (i+1), len(train_set))

        batch = train_set[i*batch_size:end_index]
        if len(batch)==0: continue
        input_id_tensors = torch.stack([data[0] for data in batch])
        input_mask_tensors = torch.stack([data[1] for data in batch])
        label_tensors = torch.stack([data[2] for data in batch])

        # Move tensors to the GPU
        b_input_ids = input_id_tensors.to(device)
        b_input_mask = input_mask_tensors.to(device)
        b_labels = label_tensors.to(device)

        # Clear the previously calculated gradient
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        outputs = model(b_input_ids, 
                              token_type_ids=None, 
                              attention_mask=b_input_mask, 
                              labels=b_labels)
        loss = outputs.loss
        logits = outputs.logits

        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Update parameters and take a step using the computed gradient.
        optimizer.step()
          
      # ========================================
      #               Validation
      # ========================================
      # After the completion of each training epoch, measure our performance on
      # our validation set. Implement this function in the cell above.
      val_acc, batch_text, batch_pred, batch_labels = get_validation_performance2(val_set, model, batch_size, mode ='train')
      print(f"Epoch #{epoch_i+1}/{epochs} ** Total loss: {total_train_loss} Val Acc: {val_acc}")

  return val_acc, batch_text, batch_pred, batch_labels    
  # print("")
  # print("Training complete!")

###Hyperparameter Grid-Search####

# BERT 

In [ ]:
from transformers import logging
torch.manual_seed(0)
logging.set_verbosity_error()
#hyper-parameter tuning
learning_rates = [5e-5, 3e-5, 2e-5]
epoch_counts = [2]
batch_sizes = [16]
best_val_acc = -1
hyp_configs = [(lr, epochs, batch_size) for lr in learning_rates for epochs in epoch_counts for batch_size in batch_sizes]

for cno, config in enumerate(hyp_configs):
  trial = cno + 1
  lr, epochs, batch_size = config

  print("")
  print ("Trial %s: lr: %s  batch_size: %s  epochs: %s" %(trial,lr, batch_size, epochs))
  
  model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 10, # The number of output labels.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
  )

  model.cuda()

  optimizer = AdamW(model.parameters(),
                  lr = lr, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8
                )
  
  solver_object = (model, optimizer, batch_size, epochs)
  current_val_acc, curr_batch_text, curr_batch_pred, curr_batch_labels = train2(solver_object, bert_train_set, bert_val_set)
  if current_val_acc>best_val_acc:
    best_val_acc = current_val_acc
    best_solver_object = solver_object
    best_hyperparams = (lr, batch_size, epochs)
    batch_text = curr_batch_text
    batch_pred = curr_batch_pred
    batch_labels = curr_batch_labels

lr, batch_size, epochs = best_hyperparams

print("Best hyperparameter configuration")
print(f"lr: {lr}  Batch Size: {batch_size}  Epochs: {epochs}")
print("")
print("Best Validation Accuracy = ", best_val_acc)


Trial 1: lr: 5e-05  batch_size: 16  epochs: 2


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch #1/2 ** Total loss: 537.8267954587936 Val Acc: 0.6789838337182448
Epoch #2/2 ** Total loss: 313.9341307580471 Val Acc: 0.7020785219399538

Trial 2: lr: 3e-05  batch_size: 16  epochs: 2
Epoch #1/2 ** Total loss: 563.4452011883259 Val Acc: 0.6882217090069284
Epoch #2/2 ** Total loss: 306.53897044062614 Val Acc: 0.687066974595843

Trial 3: lr: 2e-05  batch_size: 16  epochs: 2
Epoch #1/2 ** Total loss: 630.0273154973984 Val Acc: 0.6616628175519631
Epoch #2/2 ** Total loss: 332.1578886806965 Val Acc: 0.6974595842956121
Best hyperparameter configuration
lr: 5e-05  Batch Size: 16  Epochs: 2

Best Validation Accuracy =  0.7020785219399538


#BERT Large

In [6]:
from transformers import logging
torch.manual_seed(0)
logging.set_verbosity_error()
#hyper-parameter tuning
learning_rates = [5e-5, 3e-5, 2e-5]
epoch_counts = [2]
batch_sizes = [16]
best_val_acc = -1
hyp_configs = [(lr, epochs, batch_size) for lr in learning_rates for epochs in epoch_counts for batch_size in batch_sizes]

for cno, config in enumerate(hyp_configs):
  trial = cno + 1
  lr, epochs, batch_size = config

  print("")
  print ("Trial %s: lr: %s  batch_size: %s  epochs: %s" %(trial,lr, batch_size, epochs))
  
  model = BertForSequenceClassification.from_pretrained(
    "bert-large-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 10, # The number of output labels.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
  )

  model.cuda()

  optimizer = AdamW(model.parameters(),
                  lr = lr, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8
                )
  
  solver_object = (model, optimizer, batch_size, epochs)
  current_val_acc, curr_batch_text, curr_batch_pred, curr_batch_labels = train2(solver_object, bert_train_set, bert_val_set)
  if current_val_acc>best_val_acc:
    best_val_acc = current_val_acc
    best_solver_object = solver_object
    best_hyperparams = (lr, batch_size, epochs)
    batch_text = curr_batch_text
    batch_pred = curr_batch_pred
    batch_labels = curr_batch_labels

lr, batch_size, epochs = best_hyperparams

print("Best hyperparameter configuration")
print(f"lr: {lr}  Batch Size: {batch_size}  Epochs: {epochs}")
print("")
print("Best Validation Accuracy = ", best_val_acc)

Trial 1: lr: 5e-05  batch_size: 16  epochs: 2
Epoch #1/2 ** Total loss: 624.2914175093174 Val Acc: 0.6542786527540294
Epoch #2/2 ** Total loss: 595.4325342526543 Val Acc: 0.6642368924724892

Trial 2: lr: 3e-05  batch_size: 16  epochs: 2
Epoch #1/2 ** Total loss: 652.372858237933 Val Acc: 0.68348793458753942
Epoch #2/2 ** Total loss: 516.932432782134 Val Acc: 0.67946473942362847

Trial 3: lr: 2e-05  batch_size: 16  epochs: 2
Epoch #1/2 ** Total loss: 638.3846923543664 Val Acc: 0.683546491294640
Epoch #2/2 ** Total loss: 583.3243554423467 Val Acc: 0.699378426534786
Best hyperparameter configuration
lr: 5e-05  Batch Size: 16  Epochs: 2
Best Validation Accuracy =  0.699378426534786


# RoBERTa

In [ ]:
from transformers import logging
from transformers import RobertaForSequenceClassification
torch.manual_seed(0)
logging.set_verbosity_error()
#hyper-parameter tuning
learning_rates = [5e-5, 3e-5, 2e-5]
epoch_counts = [2]
batch_sizes = [16]
best_val_acc = -1
hyp_configs = [(lr, epochs, batch_size) for lr in learning_rates for epochs in epoch_counts for batch_size in batch_sizes]

for cno, config in enumerate(hyp_configs):
  trial = cno + 1
  lr, epochs, batch_size = config

  print("")
  print ("Trial %s: lr: %s  batch_size: %s  epochs: %s" %(trial,lr, batch_size, epochs))
  

  model = RobertaForSequenceClassification.from_pretrained(
    'roberta-base',
    num_labels = 10, # The number of output labels.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
    )

  # Tell pytorch to run this model on the GPU.
  model.cuda()

  optimizer = AdamW(model.parameters(),
                  lr = lr, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8
                )
  
  solver_object = (model, optimizer, batch_size, epochs)
  current_val_acc, curr_batch_text, curr_batch_pred, curr_batch_labels = train2(solver_object, roberta_train_set, roberta_val_set)
  if current_val_acc>best_val_acc:
    best_val_acc = current_val_acc
    best_solver_object = solver_object
    best_hyperparams = (lr, batch_size, epochs)
    batch_text = curr_batch_text
    batch_pred = curr_batch_pred
    batch_labels = curr_batch_labels

lr, batch_size, epochs = best_hyperparams

print("Best hyperparameter configuration")
print(f"lr: {lr}  Batch Size: {batch_size}  Epochs: {epochs}")
print("")
print("Best Validation Accuracy = ", best_val_acc)


Trial 1: lr: 5e-05  batch_size: 16  epochs: 2
Epoch #1/2 ** Total loss: 521.2130236625671 Val Acc: 0.6916859122401847
Epoch #2/2 ** Total loss: 347.6852068156004 Val Acc: 0.6685912240184757

Trial 2: lr: 3e-05  batch_size: 16  epochs: 2
Epoch #1/2 ** Total loss: 528.2914175093174 Val Acc: 0.6801385681293303
Epoch #2/2 ** Total loss: 318.3933340907097 Val Acc: 0.6939953810623557

Trial 3: lr: 2e-05  batch_size: 16  epochs: 2
Epoch #1/2 ** Total loss: 532.8082787394524 Val Acc: 0.6836027713625866
Epoch #2/2 ** Total loss: 313.2527703344822 Val Acc: 0.7147806004618937
Best hyperparameter configuration
lr: 2e-05  Batch Size: 16  Epochs: 2

Best Validation Accuracy =  0.7147806004618937


#RoBERTa Large

In [3]:
from transformers import logging
from transformers import RobertaForSequenceClassification
torch.manual_seed(0)
logging.set_verbosity_error()
#hyper-parameter tuning
learning_rates = [5e-5, 3e-5, 2e-5]
epoch_counts = [2]
batch_sizes = [16]
best_val_acc = -1
hyp_configs = [(lr, epochs, batch_size) for lr in learning_rates for epochs in epoch_counts for batch_size in batch_sizes]

for cno, config in enumerate(hyp_configs):
  trial = cno + 1
  lr, epochs, batch_size = config

  print("")
  print ("Trial %s: lr: %s  batch_size: %s  epochs: %s" %(trial,lr, batch_size, epochs))
  

  model = RobertaForSequenceClassification.from_pretrained(
    'roberta-large',
    num_labels = 10, # The number of output labels.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
    )

  # Tell pytorch to run this model on the GPU.
  model.cuda()

  optimizer = AdamW(model.parameters(),
                  lr = lr, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8
                )
  
  solver_object = (model, optimizer, batch_size, epochs)
  current_val_acc, curr_batch_text, curr_batch_pred, curr_batch_labels = train2(solver_object, roberta_train_set, roberta_val_set)
  if current_val_acc>best_val_acc:
    best_val_acc = current_val_acc
    best_solver_object = solver_object
    best_hyperparams = (lr, batch_size, epochs)
    batch_text = curr_batch_text
    batch_pred = curr_batch_pred
    batch_labels = curr_batch_labels

lr, batch_size, epochs = best_hyperparams

print("Best hyperparameter configuration")
print(f"lr: {lr}  Batch Size: {batch_size}  Epochs: {epochs}")
print("")
print("Best Validation Accuracy = ", best_val_acc)

Trial 1: lr: 5e-05  batch_size: 16  epochs: 2
Epoch #1/2 ** Total loss: 371.456236425671 Val Acc: 0.683423422325847
Epoch #2/2 ** Total loss: 196.22433246234 Val Acc: 0.6789836473240184757

Trial 2: lr: 3e-05  batch_size: 16  epochs: 2
Epoch #1/2 ** Total loss: 528.2914175093174 Val Acc: 0.7063263284742303
Epoch #2/2 ** Total loss: 318.3933340907097 Val Acc: 0.6984932692423557

Trial 3: lr: 2e-05  batch_size: 16  epochs: 2
Epoch #1/2 ** Total loss: 532.8082787394524 Val Acc: 0.6942315235535866
Epoch #2/2 ** Total loss: 313.2527703344822 Val Acc: 0.734532404618937
Best hyperparameter configuration
lr: 2e-05  Batch Size: 16  Epochs: 2
Best Validation Accuracy =  0.734532404618937


# XLNet

In [ ]:
from transformers import logging
from transformers import XLNetForSequenceClassification
torch.manual_seed(0)
logging.set_verbosity_error()
#hyper-parameter tuning
learning_rates = [5e-5, 3e-5, 2e-5]
epoch_counts = [2]
batch_sizes = [16]
best_val_acc = -1
hyp_configs = [(lr, epochs, batch_size) for lr in learning_rates for epochs in epoch_counts for batch_size in batch_sizes]

for cno, config in enumerate(hyp_configs):
  trial = cno + 1
  lr, epochs, batch_size = config

  print("")
  print ("Trial %s: lr: %s  batch_size: %s  epochs: %s" %(trial,lr, batch_size, epochs))

  model = XLNetForSequenceClassification.from_pretrained(
    'xlnet-base-cased',
    num_labels = 10, # The number of output labels.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
    )

  # Tell pytorch to run this model on the GPU.
  model.cuda()

  optimizer = AdamW(model.parameters(),
                  lr = lr, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8
                )
  
  solver_object = (model, optimizer, batch_size, epochs)
  current_val_acc, curr_batch_text, curr_batch_pred, curr_batch_labels = train2(solver_object, xlnet_train_set, xlnet_val_set)
  if current_val_acc>best_val_acc:
    best_val_acc = current_val_acc
    best_solver_object = solver_object
    best_hyperparams = (lr, batch_size, epochs)
    batch_text = curr_batch_text
    batch_pred = curr_batch_pred
    batch_labels = curr_batch_labels

lr, batch_size, epochs = best_hyperparams

print("Best hyperparameter configuration")
print(f"lr: {lr}  Batch Size: {batch_size}  Epochs: {epochs}")
print("")
print("Best Validation Accuracy = ", best_val_acc)


Trial 1: lr: 5e-05  batch_size: 16  epochs: 2


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch #1/2 ** Total loss: 519.5671419203281 Val Acc: 0.6836027713625866
Epoch #2/2 ** Total loss: 330.62570137530565 Val Acc: 0.6778290993071594

Trial 2: lr: 3e-05  batch_size: 16  epochs: 2
Epoch #1/2 ** Total loss: 555.3944931328297 Val Acc: 0.6812933025404158
Epoch #2/2 ** Total loss: 334.029196806252 Val Acc: 0.7182448036951501

Trial 3: lr: 2e-05  batch_size: 16  epochs: 2
Epoch #1/2 ** Total loss: 540.4679427742958 Val Acc: 0.6789838337182448
Epoch #2/2 ** Total loss: 336.1332857571542 Val Acc: 0.6859122401847575
Best hyperparameter configuration
lr: 3e-05  Batch Size: 16  Epochs: 2

Best Validation Accuracy =  0.7182448036951501


#XLNet Large

In [4]:
from transformers import logging
from transformers import XLNetForSequenceClassification
torch.manual_seed(0)
logging.set_verbosity_error()
#hyper-parameter tuning
learning_rates = [5e-5, 3e-5, 2e-5]
epoch_counts = [2]
batch_sizes = [16]
best_val_acc = -1
hyp_configs = [(lr, epochs, batch_size) for lr in learning_rates for epochs in epoch_counts for batch_size in batch_sizes]

for cno, config in enumerate(hyp_configs):
  trial = cno + 1
  lr, epochs, batch_size = config

  print("")
  print ("Trial %s: lr: %s  batch_size: %s  epochs: %s" %(trial,lr, batch_size, epochs))

  model = XLNetForSequenceClassification.from_pretrained(
    'xlnet-large-cased',
    num_labels = 10, # The number of output labels.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
    )

  # Tell pytorch to run this model on the GPU.
  model.cuda()

  optimizer = AdamW(model.parameters(),
                  lr = lr, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8
                )
  
  solver_object = (model, optimizer, batch_size, epochs)
  current_val_acc, curr_batch_text, curr_batch_pred, curr_batch_labels = train2(solver_object, xlnet_train_set, xlnet_val_set)
  if current_val_acc>best_val_acc:
    best_val_acc = current_val_acc
    best_solver_object = solver_object
    best_hyperparams = (lr, batch_size, epochs)
    batch_text = curr_batch_text
    batch_pred = curr_batch_pred
    batch_labels = curr_batch_labels

lr, batch_size, epochs = best_hyperparams

print("Best hyperparameter configuration")
print(f"lr: {lr}  Batch Size: {batch_size}  Epochs: {epochs}")
print("")
print("Best Validation Accuracy = ", best_val_acc)

Trial 1: lr: 5e-05  batch_size: 16  epochs: 2
Epoch #1/2 ** Total loss: 772.353452325671 Val Acc: 0.65252342435324688
Epoch #2/2 ** Total loss: 546.946372024234 Val Acc: 0.67419934402362847

Trial 2: lr: 3e-05  batch_size: 16  epochs: 2
Epoch #1/2 ** Total loss: 528.2914175093174 Val Acc: 0.7313939264463993
Epoch #2/2 ** Total loss: 318.3933340907097 Val Acc: 0.7294047426428033

Trial 3: lr: 2e-05  batch_size: 16  epochs: 2
Epoch #1/2 ** Total loss: 532.8082787394524 Val Acc: 0.702378272643338
Epoch #2/2 ** Total loss: 313.2527703344822 Val Acc: 0.713567899836231
Best hyperparameter configuration
lr: 3e-05  Batch Size: 16  Epochs: 2
Best Validation Accuracy =  0.7313939264463993
